In [1]:
from z3 import *
import numpy as np
from itertools import combinations
from typing import Sequence
from tqdm.notebook import tqdm

Read instance file:

In [2]:
input_filename = '../../Instances/16x16-ide.txt'

w, h, n, DX, DY = None, None, None, None, None

with open(input_filename, 'r') as f_in:
    lines = f_in.read().splitlines()

    split = lines[0].split(' ')
    w = int(split[0])
    h = int(split[1])

    n = int(lines[1])

    DX = []
    DY = []

    for i in range(int(n)):
        split = lines[i + 2].split(' ')
        DX.append(int(split[0]))
        DY.append(int(split[1]))

In [3]:
indexes = {}
for idx, dim in enumerate(zip(DX, DY)):
    if dim in indexes:
        indexes[dim] += [idx]
    else:
        indexes[dim] = [idx]

print(indexes)

{(3, 3): [0], (3, 4): [1, 3, 6, 9], (3, 5): [2], (3, 6): [4], (3, 7): [5], (3, 8): [7], (3, 10): [8], (4, 7): [10], (7, 9): [11]}


Solver:

In [4]:
solver = Solver()

Model:

In [5]:
B = [[[Bool(f'B_{i}_{j}_{k}') for k in range(n)] for j in range(w)] for i in range(h)]

In [6]:
def at_least_one(bool_vars: Sequence):
    return Or(bool_vars)

def at_most_one(bool_vars: Sequence):
    return [Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)]

In [7]:
# Constraint "at most one piece"
for i in range(h):
    for j in range(w):
        solver.add(at_most_one(B[i][j]))

In [8]:
def is_valid(i1, j1, i2, j2, dx, dy):
    right = (j2 >= j1 + dx) and (i2 <= i1)
    up = (i2 <= i1 - dy) and (j2 >=j1)
    return right or up

# Iterate over all the dimensions
for (dx, dy), p_list in indexes.items():
    if len(p_list) > 1:  # case 1: multiple identical pieces
        # Iterate over identical pieces in pairs
        for i in range(len(p_list) - 1):
            p1 = p_list[i]
            p2 = p_list[i+1]

            package_clauses_p1 = {}
            package_clauses_p2 = {}
            package_clauses_joint = []

            # Iterate over all the coordinates where p can fit
            for i in range(h - dy + 1):
                for j in range(w - dx + 1):

                    patch_clauses_p1 = []
                    patch_clauses_p2 = []
                    # Iterate over the cells of p's patch
                    for f1 in range(dy):
                        for f2 in range(dx):
                            patch_clauses_p1.append(B[i + f1][j + f2][p1])
                            patch_clauses_p2.append(B[i + f1][j + f2][p2])
                    
                    # (i+dy-1, j) bottom-left corner
                    package_clauses_p1[(i + dy - 1, j)] = And(patch_clauses_p1)
                    package_clauses_p2[(i + dy - 1, j)] = And(patch_clauses_p2)
            
            # Filter valid p2 clauses
            for (i1, j1), patch_p1 in package_clauses_p1.items():
                # Condition for validity: i2 <= i1 and j2 >= j1
                valid_patches_p2 = [patch_p2 for (i2, j2), patch_p2 in package_clauses_p2.items() if is_valid(i1, j1, i2, j2, dx, dy)]
                
                package_clauses_joint.append(And(patch_p1, at_least_one(valid_patches_p2)))
                
            solver.add(at_least_one(package_clauses_joint))
    else:  # case 2: unique piece
        p = p_list[0]
        package_clauses = []

        # Iterate over all the coordinates where p can fit
        for i in range(h - dy + 1):
            for j in range(w - dx + 1):

                patch_clauses = []
                # Iterate over the cells of p's patch
                for f1 in range(dy):
                    for f2 in range(dx):
                        patch_clauses.append(B[i + f1][j + f2][p])

                package_clauses.append(And(patch_clauses))

        solver.add(at_least_one(package_clauses))

In [9]:
%%time
solver.check()

CPU times: user 3.03 s, sys: 7.48 ms, total: 3.04 s
Wall time: 3.06 s


sat

From Z3 model solution to file:

In [10]:
solution = np.zeros((h, w, n), dtype=bool)
model = solver.model()

for i in range(h):
    for j in range(w):
        for k in range(n):
            solution[i, j, k] = is_true(model[B[i][j][k]])

In [11]:
xy = {}
for p in range(n):
    y_ids, x_ids = solution[:, :, p].nonzero()
    #print(solution[:, :, p])
    x = np.min(x_ids)
    y = h-1-np.max(y_ids)
    xy[p] = [x, y]

In [12]:
xy

{0: [0, 8],
 1: [6, 0],
 2: [0, 11],
 3: [6, 4],
 4: [3, 0],
 5: [13, 9],
 6: [6, 8],
 7: [0, 0],
 8: [3, 6],
 9: [6, 12],
 10: [9, 9],
 11: [9, 0]}

In [13]:
output_filename = '../../pwp_utilities/16x16.txt'
with open(output_filename, 'w') as f_out:
    f_out.write('{} {}\n'.format(w, h))
    f_out.write('{}\n'.format(n))
    for i in range(n):
        f_out.write('{} {}\t{} {}\n'.format(DX[i], DY[i], xy[i][0], xy[i][1]))